In [1]:
import numpy as np
import pandas as pd
import imageio
import h5py
import pickle

from tqdm import tqdm
from xqdm import xqdm
from pathlib import Path
import matplotlib.pyplot as plt
import re

import keras.layers as L
from keras.models import Model
from keras.utils import plot_model

Using TensorFlow backend.


In [15]:
I = L.Input(shape=(6,))
x = L.Dense(32, activation='relu')(I)
x = L.Dense(64, activation='relu')(x)
x = L.Dropout(0.2)(x)
x = L.Dense(128, activation='relu')(x)
x = L.Dropout(0.2)(x)
x = L.Dense(1, activation='linear')(x)

model = Model(inputs=I, outputs=x)

In [16]:
model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 6)                 0         
_________________________________________________________________
dense_9 (Dense)              (None, 32)                224       
_________________________________________________________________
dense_10 (Dense)             (None, 64)                2112      
_________________________________________________________________
dropout_5 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_11 (Dense)             (None, 128)               8320      
_________________________________________________________________
dropout_6 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 129 

In [17]:
model.compile('adam', loss='mean_squared_error')

In [18]:
with open('data/interpolation_xy.pkl', 'rb') as f:
    datasets = pickle.load(f)

In [59]:
def make_dataset(d, col):
    X_train, X_test, Y_train, Y_test = [], [], [], []
    for j, dataset in enumerate(datasets):
        x = dataset[0]['input_orientation_' + col].astype(np.float32)
        y = dataset[1]['input_orientation_' + col].astype(np.float32)
        for i in range(len(x) - d):
            if i < len(x) // 2:
                X_train.append(x[i:i+6])
                Y_train.append(y[i+d])
            else:
                X_test.append(x[i:i+6])
                Y_test.append(y[i+d])

    X_train = np.stack(X_train, axis=0)
    X_test = np.stack(X_test, axis=0)
    Y_train = np.stack(Y_train, axis=0)
    Y_test = np.stack(Y_test, axis=0)

    #print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

    with open(f'data/mlp{(d//6)*100}_{col}.pkl', 'wb') as f:
        pickle.dump((X_train, X_test, Y_train, Y_test), f)

In [60]:
def __make_dataset(args):
    make_dataset(*args)

In [61]:
items = [
    (6, 'yaw'),
    (6, 'pitch'),
    (6, 'roll'),
    (12, 'yaw'),
    (12, 'pitch'),
    (12, 'roll'),
    (18, 'yaw'),
    (18, 'pitch'),
    (18, 'roll'),
]

In [62]:
_ = xqdm(__make_dataset, items, ncols=100)

100%|█████████████████████████████████████████████████████████████████| 9/9 [00:35<00:00,  3.98s/it]


[None, None, None, None, None, None, None, None, None]